# 2.4a & 2.4b Hacks
- toc: true 
- badges: true
- comments: true
- categories: [markdown]

In [68]:
from flask import Flask
from flask_sqlalchemy import SQLAlchemy

"""
These object and definitions are used throughout the Jupyter Notebook.
"""

# Setup of key Flask object (app)
app = Flask(__name__)
# Setup SQLAlchemy object and properties for the database (db)
database = 'sqlite:///sqlitebbal.db'  # path and filename of database
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
app.config['SQLALCHEMY_DATABASE_URI'] = database
app.config['SECRET_KEY'] = 'SECRET_KEY'
db = SQLAlchemy()


# This belongs in place where it runs once per project
db.init_app(app)

In [69]:
import datetime
from datetime import datetime
import json

from sqlalchemy.exc import IntegrityError
from werkzeug.security import generate_password_hash, check_password_hash


class User(db.Model):
    __tablename__ = 'users'  # table name is plural, class name is singular

    id = db.Column(db.Integer, primary_key=True)
    _name = db.Column(db.String(255), unique=False, nullable=False)
    _hitrate = db.Column(db.String(255), unique=True, nullable=False)

    
    def __init__(self, name, hitrate):

        self._name = name    # variables with self prefix become part of the object, 
        self._hitrate = hitrate

    @property
    def name(self):
        return self._name
    
    # a setter function, allows name to be updated after initial object creation
    @name.setter
    def name(self, name):
        self._name = name
    
    # a getter method, extracts uid from object
    @property
    def hitrate(self):
        return self._hitrate
    
    # a setter function, allows uid to be updated after initial object creation
    @hitrate.setter
    def uid(self, hitrate):
        self._hitrate = hitrate
        


    def __str__(self):
        return json.dumps(self.read())

    # CRUD create/add a new record to the table
    # returns self or None on error
    def create(self):
        try:
            # creates a person object from User(db.Model) class, passes initializers
            db.session.add(self)  # add prepares to persist person object to Users table
            db.session.commit()  # SqlAlchemy "unit of work pattern" requires a manual commit
            return self
        except IntegrityError:
            db.session.remove()
            return None

    # CRUD read converts self to dictionary
    # returns dictionary
    def read(self):
        return {
            "id": self.id,
            "name": self.name,
            "hitrate": self.hitrate,
        }

    # CRUD update: updates user name, password, phone
    # returns self
    def update(self, name="", hitrate=""):
        self.name = name
        db.session.commit()
        return self

    # CRUD delete: remove self
    # None
    def delete(self):
        db.session.delete(self)
        db.session.commit()
        return None
    # just a definition

In [70]:
def initUsers():
    with app.app_context():
        db.create_all()

        u1 = User(name='Zach LaVine OVER Assists', hitrate="60%")
        u2 = User(name='Kelly Oubre Jr OVER Points', hitrate="80%")
        u3 = User(name='Joel Embiid OVER 1H PRA', hitrate="100%")
        u4 = User(name='James Harden OVER Points', hitrate="20%")
        u5 = User(name='Mitchell Robinson OVER Points', hitrate="40%")
        u6 = User(name='Jalen Bruson UNDER 1H PRA', hitrate="0%")
        users = [u1, u2, u3, u4, u5, u6]

        """Builds sample user/note(s) data"""
        for user in users:
            try:
                '''add user to table'''
                object = user.create()
                print(f"Created new uid {object.uid}")
            except:  # error raised if object nit created
                '''fails with bad or duplicate data'''
                print(f"Records exist uid {user.uid}, or error.")
                
initUsers()

Records exist uid 60%, or error.
Records exist uid 80%, or error.
Records exist uid 100%, or error.
Records exist uid 20%, or error.
Records exist uid 40%, or error.
Records exist uid 0%, or error.


In [71]:
def read():
    with app.app_context():
        table = User.query.all()
    json_ready = [user.read() for user in table] # "List Comprehensions", for each user add user.read() to list
    return json_ready

read()

[{'id': 4, 'name': 'James Harden OVER Points', 'hitrate': '20%'},
 {'id': 5, 'name': 'Mitchell Robinson OVER Points', 'hitrate': '40%'},
 {'id': 6, 'name': 'Jalen Bruson UNDER 1H PRA', 'hitrate': '0%'},
 {'id': 10, 'name': 'Zach LaVine OVER Assists', 'hitrate': '60%'},
 {'id': 11, 'name': 'Kelly Oubre Jr OVER Points', 'hitrate': '80%'},
 {'id': 13, 'name': 'Joel Embiid OVER 1H PRA', 'hitrate': '100%'}]

In [72]:
def find_by_name(name):
    with app.app_context():
        user = User.query.filter_by(_name=name).first()
    return user # returns user object

def check_credentials(uid, hitrate):
    # query email and return user record
    user = find_by_name(uid)
    if user == 1:
        return False
    if hitrate == "60%":
        return True
    return True
        
check_credentials("1", "60")
    

True

In [73]:
def deleteuser():
    id = input("enter name")
    user = find_by_name(id)
    print(user)
    if user is not None:
        with app.app_context():
            db.session.delete(user)  
            db.session.commit()
        print("User deleted.")
    else:
        print("User not found.")
    
deleteuser()
print("new DB")
read()

{"id": 4, "name": "James Harden OVER Points", "hitrate": "20%"}
User deleted.
new DB


[{'id': 5, 'name': 'Mitchell Robinson OVER Points', 'hitrate': '40%'},
 {'id': 6, 'name': 'Jalen Bruson UNDER 1H PRA', 'hitrate': '0%'},
 {'id': 10, 'name': 'Zach LaVine OVER Assists', 'hitrate': '60%'},
 {'id': 11, 'name': 'Kelly Oubre Jr OVER Points', 'hitrate': '80%'},
 {'id': 13, 'name': 'Joel Embiid OVER 1H PRA', 'hitrate': '100%'}]

In [77]:
def updateuser():
    
    id2 = input("id")
    name2= input("new name")
    with app.app_context():
        user = User.query.filter_by(id=id2).first()
        user.update(name=name2)
        db.session.commit()

def read():
    with app.app_context():
        table = User.query.all()
    json_ready = [user.read() for user in table] # "List Comprehensions", for each user add user.read() to list
    
    return json_ready

read()


updateuser()

In [75]:
def create():
    # optimize user time to see if name exists
    name = input("Enter your user id:")
    user = find_by_name(name)
    try:
        print("Found\n", user.read())
        return
    except:
        pass # keep going
    
    # request value that ensure creating valid object
    name = input("Enter your name:")
    hitrate = input("Enter your hitrate")
    
    # Initialize User object before date
    user = User(name=name,  
                hitrate=hitrate
                )
           
    # write object to database
    with app.app_context():
        try:
            object = user.create()
            print("Created\n", object.read())
        except:  # error raised if object not created
            print("Unknown error name {name}")
        
create()


Created
 {'id': 14, 'name': 'James Harden OVER Points', 'hitrate': '20%'}


# Menu 

In [78]:
def process():
    endpoint = input("read, delete, update, create")
    if endpoint == "read":
        with app.app_context():
            table = User.query.all()
            json_ready = [user.read() for user in table] # "List Comprehensions", for each user add user.read() to list
        return json_ready
    elif endpoint == "delete":
        deleteuser()
    elif endpoint == "update":
        updateuser()
    elif endpoint == "create":
        create()
    else:
        print('function not found')


process()

[{'id': 5, 'name': 'Mitchell Robinson OVER Points', 'hitrate': '40%'},
 {'id': 6, 'name': 'Jalen Bruson UNDER 1H PRA', 'hitrate': '0%'},
 {'id': 10, 'name': 'Zach LaVine OVER Assists', 'hitrate': '60%'},
 {'id': 11, 'name': 'Kelly Oubre Jr OVER Points', 'hitrate': '80%'},
 {'id': 13, 'name': 'Joel Embiid OVER 1H PRA', 'hitrate': '100%'},
 {'id': 14, 'name': 'James Harden OVER Points', 'hitrate': '20%'}]